In [14]:
import pandas as pd
import numpy as np
import geopandas as gp
import geog
import shapely.geometry
import warnings
warnings.filterwarnings("ignore")
import datetime

In [15]:
shapely.__version__

'1.8.0'

In [16]:
location = pd.read_csv('deduped_merged_data.csv')
real_estate = pd.read_csv('ResidentialModelBuilding/mash_w_coord.csv')

In [17]:
location.head()

,advertiser_id,latitude,longitude,horizontal_accuracy,location_at,venue_name,venue_category,speed,publisher_id,dwell_time,...,pct_65over,no_health_under65,civilian_adults_total,pct_civadults_below_povertylvl,disabilities_18to64,disabilities_65&over,OneOrMore_PersonPerRoom_Owner,OneOrMore_PersonPerRoom_Renter,pct_males,pct_25andOver_NoHighSchoolDiploma
0,4B56BE3D-EECF-4F0D-B15D-7F1FF9F0D17F,38.848632,-77.099073,800.0,1554060225,NaN,NaN,0.0,NaN,NaN,...,0.042051,0.022564,871,0.063146,0.0,1.0,0.0,0.0,0.566154,0.0
1,4B56BE3D-EECF-4F0D-B15D-7F1FF9F0D17F,38.848273,-77.098185,15.9,1554541597,NaN,NaN,0.0,NaN,345000.0,...,0.042051,0.022564,871,0.063146,0.0,1.0,0.0,0.0,0.566154,0.0
2,4B56BE3D-EECF-4F0D-B15D-7F1FF9F0D17F,38.848270,-77.098714,800.0,1557635013,NaN,NaN,0.0,NaN,NaN,...,0.042051,0.022564,871,0.063146,0.0,1.0,0.0,0.0,0.566154,0.0
3,4B56BE3D-EECF-4F0D-B15D-7F1FF9F0D17F,38.848954,-77.099073,800.0,1559109705,NaN,NaN,0.0,NaN,NaN,...,0.042051,0.022564,871,0.063146,0.0,1.0,0.0,0.0,0.566154,0.0
4,4B56BE3D-EECF-4F0D-B15D-7F1FF9F0D17F,38.848954,-77.099073,800.0,1560681834,NaN,NaN,0.0,NaN,NaN,...,0.042051,0.022564,871,0.063146,0.0,1.0,0.0,0.0,0.566154,0.0


In [18]:
len(real_estate)

118707

In [19]:
class ConnectMobileWithRealEstate():
    def __init__(self, mobile_df, count_once_per_day=False):
        self.mobile_df = mobile_df
        self.count_once_per_day = count_once_per_day #counted only once per day if they remain in the same stop
        
    def calculate_stats(self, mobile_df):
        
        mobile_df = mobile_df[mobile_df['dwell_start'].notnull() & mobile_df['dwell_end'].notnull()].reset_index(drop=True)
        
        mobile_df['total_commuting'] = 0
        mobile_df.loc[mobile_df.place_id==-1,'total_commuting'] = 1
        
        mobile_df['residents'] = 0
        mobile_df.loc[mobile_df.census_block_group==mobile_df.census_block_group_stop, 'residents'] = 1
        
        mobile_df['people_in_area'] = 1
        
        mobile_df['dwell_estimated'] = mobile_df.dwell_end - mobile_df.dwell_start
        
        mobile_df.loc[mobile_df.place_id_latitude.isna(), 'place_id_latitude'] = mobile_df.loc[mobile_df.place_id_latitude.isna(), 'latitude']
        mobile_df.loc[mobile_df.place_id_longitude.isna(), 'place_id_longitude'] = mobile_df.loc[mobile_df.place_id_longitude.isna(), 'longitude']

        return mobile_df        
        
        
    def timerange(self, x, time_unit='D'):
        r = pd.date_range(x.dwell_start, x.dwell_end,freq=time_unit)
        return [i for i in r]
        
    def group_by_time(self):
        
        mobile_df = self.calculate_stats(self.mobile_df)
        
        mobile_df['dwell_start'] = pd.to_datetime(mobile_df['dwell_start'],unit='s', errors='coerce')
        mobile_df['dwell_start'] = mobile_df['dwell_start'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

        mobile_df['dwell_end']= pd.to_datetime(mobile_df['dwell_end'],unit='s', errors='coerce')
        mobile_df['dwell_end'] = mobile_df['dwell_end'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
                
        mobile_df['time_list'] = mobile_df.apply(lambda x: self.timerange(x), axis=1)
                
        if self.count_once_per_day: # and (self.group_by == 'day_of_week')
            mobile_df.drop_duplicates(['advertiser_id','dwell_start','time_unit'], inplace=True, ignore_index=True)
                        
        mobile_df = mobile_df.drop(['census_block_group', 'census_block_group_stop','latitude', 'longitude', 'dwell_start', 'dwell_end', 'place_id'], axis=1)
        self.mobile_df = gp.GeoDataFrame(mobile_df, geometry=gp.points_from_xy(mobile_df.place_id_longitude, mobile_df.place_id_latitude), crs = 'epsg:4326')

        
    def merge_datasets(self, real_estate_df, distance=1000, time_group = 'day_of_week', only_non_residents=True, empty_area_impute_strat = 'mean'):
        
        real_estate_df = real_estate_df.copy()
        gmobile = self.mobile_df.copy()
        
        min_lat = gmobile.place_id_latitude.min()
        max_lat = gmobile.place_id_latitude.max()
        min_lon = gmobile.place_id_longitude.min()
        max_lon = gmobile.place_id_longitude.max()
        
        real_estate_df = real_estate_df.loc[(real_estate_df.Latitude>= min_lat) & (real_estate_df.Latitude<= max_lat) & (real_estate_df.Longitude >= min_lon) & (real_estate_df.Longitude <= max_lon)].reset_index(drop=True)
        
        if only_non_residents:
        
            columns_toNull = [i for i in gmobile.columns if i not in ['advertiser_id','place_id_latitude','place_id_longitude','residents', 'hour_of_week', 'hour_of_day', 'day_of_week','people_in_area', 'time_list','geometry']]
            gmobile.loc[gmobile.residents==1, columns_toNull] = np.nan 
        
        gmobile.reset_index(inplace=True)
        gmobile.rename({'index':'mobile_index'}, axis=1, inplace=True)
        gmobile.drop(['place_id_latitude','place_id_longitude'], axis=1, inplace=True)
        
        num_points = 50
        radius = distance  #radius in meters
        angles = np.linspace(0, 360, num_points)
        
        real_estate_df.rename({'CensusBlockGroup':'census_block_group'}, axis=1, inplace=True)
        real_estate_df = real_estate_df.reset_index()
        greal = gp.GeoDataFrame(real_estate_df, geometry=gp.points_from_xy(real_estate_df.Longitude, real_estate_df.Latitude), crs = 'epsg:4326')
        
        for idx, point in greal.geometry.items():
            polygon_array = geog.propagate(point, angles, radius)
            circle = shapely.geometry.Polygon(polygon_array)
            greal.loc[idx, 'geometry'] = circle
        print('done with loop')
        
        property_columns = [i for i in greal.columns if i != 'geometry']
        column_operations = {i:'mean' for i in gmobile.columns if i not in ['census_block_group', 'hour_of_week', 'hour_of_day', 'day_of_week','day_of_year','people_in_area','index_right','geometry', 'advertiser_id', 'time_list','mobile_index','residents']}
        column_operations = {**column_operations, **{'people_in_area':'sum', 'residents':'sum'}}
        
        merged = gp.sjoin(greal[['geometry','index']], gmobile[['mobile_index','geometry']], predicate='contains', how='inner').drop(['index_right','geometry'], axis=1).reset_index(drop=True)
        #could weight by dwell in case of both or group by days 
        print('done with merge')
        merged = merged.merge(real_estate_df, on = 'index').merge(gmobile, on='mobile_index')
        merged.drop('mobile_index', axis=1, inplace=True)
        
        merged
        merged = merged.explode('time_list')        
        
        if len(merged)==0:
            return merged
            
        merged['day_of_week'] = merged.time_list.dt.dayofweek
        merged['hour_of_day'] = merged.time_list.dt.hour
        merged['hour_of_week'] = merged.day_of_week * 24 + merged.hour_of_day
        merged['day_of_year'] = merged.time_list.dt.dayofyear
        
        merged.drop('time_list', axis=1, inplace=True)
        
        print('done with explode')
        
        if time_group == 'day_of_week':
            merged.people_in_area = merged.people_in_area / merged.groupby(property_columns + ['day_of_year'] +['advertiser_id']).people_in_area.transform('sum')
            merged.residents = merged.residents / merged.groupby(property_columns + ['day_of_year'] +['advertiser_id']).residents.transform('sum')

        merged.drop('advertiser_id', axis=1, inplace=True)
        
        merged = merged.groupby(property_columns + [time_group]).agg(column_operations).reset_index()
        final = merged[property_columns].drop_duplicates().set_index('index')
        
        for i in [x for x in merged[time_group].unique()]:
            sub = merged.loc[merged[time_group]==i, [i for i in merged.columns if ((i not in property_columns) | (i=='index'))]].drop(time_group, axis=1).set_index('index')
            sub.columns = [str(i) + '__' + x for x in sub.columns if x not in property_columns]
            final = final.join(sub)
        
        mobility_columns = [i for i in final.columns if '__' in i]
        final.reset_index(inplace=True)
        final.rename({'index':'property_id'}, axis =1, inplace=True)
        
        #residents should maybe be averaged?
        people_in_area_columns = [i for i in mobility_columns if i.split('__')[1] in ['residents','people_in_area']]
        final[people_in_area_columns] = final[people_in_area_columns].fillna(0)
    
        if empty_area_impute_strat == 'mean':
            new = final.copy()
        
            for i in mobility_columns:
                new.loc[final[i].isna(), i] = final.loc[final[i].isna(), [x for x in mobility_columns if x.split('__')[1]==i.split('__')[1]]].mean(axis=1)

        if empty_area_impute_strat == 'zero':
            new = final.fillna(0) 
            
        else:
            new = final.copy()
            
        return new
            

In [20]:
features_not_to_use = ['dwell_delta', 'horizontal_accuracy',
       'location_at', 'venue_name', 'venue_category', 'speed', 'publisher_id',
       'dwell_time', 'batch_id_x', 'geometry', 'index_right',
       'home_long', 'home_lat', 'batch_id_y', 'State',
       'County', 'White alone', 'Black or African American alone',
       'American Indian and Alaska Native alone', 'Asian alone',
       'Native Hawaiian and Other Pacific Islander alone',
       'Some other race alone', 'Two or more races', 'Males', 'Females',
       'Some college 1 or more years no degree', "Associate's degree",
       "Bachelor's degree", "Master's degree", 'Professional school degree',
       'Doctorate degree','owner_occupied_units',
       'nonvet_povertylvl_18to64', 'vet_povertylvl_18to64',
       'nonvet_povertylvl_65&over', 'vet_povertylvl_65&over',
       'owner_occupied_1.01to1.5_PeoplePerRoom',
       'owner_occupied_1.51to2_PeoplePerRoom',
       'owner_occupied_2.01orMore_PeoplePerRoom',
       'renter_occupied_1.01to1.5_PeoplePerRoom',
       'renter_occupied_1.5to2_PeoplePerRoom',
       'renter_occupied_2.01orMore_PeoplePerRoom','renter_occupied_units']

In [21]:
real_estate.drop(['longitude','latitude'],axis=1, inplace=True)

In [22]:
location.drop(features_not_to_use, axis=1, inplace=True)

In [23]:
ob = ConnectMobileWithRealEstate(location.sample(2000000, random_state=42))

In [24]:
len(location)

15845966

In [25]:
ob.group_by_time()

In [26]:
ob.mobile_df.head(5)

,advertiser_id,place_id_latitude,place_id_longitude,dwell_estimated,population_total,have_child_under18,total_occupied_units,Median_Household_Income,civilian_laborforce_unemployment,pct_renter_occupied,...,disabilities_65&over,OneOrMore_PersonPerRoom_Owner,OneOrMore_PersonPerRoom_Renter,pct_males,pct_25andOver_NoHighSchoolDiploma,total_commuting,residents,people_in_area,time_list,geometry
0,F1A7CE62-325C-4B3F-BFBC-E65343AC99D6,38.892000,-76.921680,42721.0,923,0.166847,313,107188.0,0.114990,0.038339,...,0.000000,0.000000,1.000000,0.551463,0.153184,0,1,1,[2019-08-24 14:51:38-04:00],POINT (-76.92168 38.89200)
1,E844E47C-BC3F-47ED-BB7E-5900A9348E19,38.851973,-77.108598,312.0,1807,0.253459,700,136316.0,0.000000,0.178571,...,0.812500,0.205217,0.160000,0.385722,0.018139,1,0,1,[2019-05-28 11:37:12-04:00],POINT (-77.10860 38.85197)
2,73B3F0BD-0A7B-42FC-BA61-848AC242CCDE,38.917710,-77.001370,20506.0,1341,0.111857,594,49130.0,0.078974,0.631313,...,0.263636,0.000000,0.000000,0.354213,0.158818,0,1,1,[2019-05-28 07:17:38-04:00],POINT (-77.00137 38.91771)
3,E4A8DFC7-09D1-45F3-A757-01AF49B2486D,38.969026,-76.985008,2633.0,1759,0.355884,491,54010.0,0.129683,0.415479,...,0.096154,0.038328,0.107843,0.471859,0.280597,1,0,1,[2019-06-11 12:46:07-04:00],POINT (-76.98501 38.96903)
4,CCB16E82-B029-4CBC-AEE0-2DD77DF15859,38.929380,-77.027395,43135.0,3605,0.088488,1345,116858.0,0.061728,0.455762,...,0.425676,0.028689,0.104405,0.510125,0.126840,0,1,1,[2019-06-08 16:19:38-04:00],POINT (-77.02740 38.92938)


In [ ]:
mobility_distance_1000_no_impute = ob.merge_datasets(real_estate, distance =500, empty_area_impute_strat = 'none')
mobility_distance_1000_no_impute

In [ ]:
#mobility_distance_1000_no_impute.to_csv('MobilityDistance/mobility_distance_1000_no_impute.csv', index=False)